---
---
# Red-Light Violation Detection
---
---

 An automated system for detecting red-light traffic violations has been developed, which could play a significant role in transportation management within smart cities. The system primarily relies on modern computer vision techniques and was implemented using the OpenCV library in a Python environment. It mainly consists of an object detector and tracker that work together to accurately maintain the positions of existing vehicles. The primary objective of the system is to identify and mark the locations of vehicles that violate traffic signals. The results demonstrated high accuracy, with all violating vehicles being detected and precisely distinguished.



#Setup Environment

In [1]:
!pip install opencv-python opencv-python-headless numpy
!pip  install ultralytics
!pip install google-colab


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 14.1 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


# Importing libraries

In [2]:
from ultralytics import YOLO, solutions
import numpy as np
import time
import cv2
import cv2
from google.colab.patches import cv2_imshow

#Load the YOLO model

In [3]:
# YOLO from v8
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:01<00:00, 46.0MB/s]


In [4]:
#The COCO dataset includes labels
coco = model.model.names
TargetLabels = [ "car", "motorcycle", "bus", "truck", "traffic light"]

In [5]:
def is_region_light(image, polygon, brightness_threshold=128):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(gray_image)
    cv2.fillPoly(mask, [np.array(polygon)], 255)
    roi = cv2.bitwise_and(gray_image, gray_image, mask=mask)
    mean_brightness = cv2.mean(roi, mask=mask)[0]
    return mean_brightness > brightness_threshold

def draw_text_with_background(frame, text, position, font, scale, text_color, background_color, border_color, thickness=2, padding=5):
    """Draw text with background and border on the frame."""
    (text_width, text_height), baseline = cv2.getTextSize(text, font, scale, thickness)
    x, y = position
    # Background rectangle
    cv2.rectangle(frame,
                  (x - padding, y - text_height - padding),
                  (x + text_width + padding, y + baseline + padding),
                  background_color,
                  cv2.FILLED)
    # Border rectangle
    cv2.rectangle(frame,
                  (x - padding, y - text_height - padding),
                  (x + text_width + padding, y + baseline + padding),
                  border_color,
                  thickness)
    # Text
    cv2.putText(frame, text, (x, y), font, scale, text_color, thickness, lineType=cv2.LINE_AA)


## Prepare the Video Capture

In [7]:
# Load the video
cap = cv2.VideoCapture('/content/videosaudiarabia_85t0Fd6o.mov')
assert cap.isOpened(), 'Error reading video file'

In [8]:
# Get video frame properties
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [9]:
#output
video_writer = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

In [10]:
print("Video Resolution: {}x{}".format(width, height))
print("Frames per second: {}".format(fps))

Video Resolution: 576x320
Frames per second: 30


In [11]:
# Define ROI (Region of Interest)
roi_bottom = (10, height - 30, width - 10, height)


In [12]:
RedLight = np.array([[998, 125],[998, 155],[972, 152],[970, 127]])
ROI = np.array([[910, 372],[388, 365],[338, 428],[917, 441]])

In [13]:
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Number of frames have finished.")
        break

    # Draw the red light and ROI regions
    cv2.polylines(frame, [RedLight], True, [0, 0, 255], 1)  # RedLight region
    cv2.polylines(frame, [ROI], True, [255, 0, 0], 2)  # Blue ROI

    # Draw the blue rectangle (same size as specified in roi_bottom)
    cv2.rectangle(frame, (roi_bottom[0], roi_bottom[1]), (roi_bottom[2], roi_bottom[3]), (255, 0, 0), 2)  # Blue box

    # Perform YOLOv8 prediction
    results = model.predict(frame, conf=0.75)
    for result in results:
        boxes = result.boxes.xyxy
        confs = result.boxes.conf
        classes = result.boxes.cls

        # Loop through the detected boxes and their associated classes
        for box, conf, cls in zip(boxes, confs, classes):
            if coco[int(cls)] in TargetLabels:
                x1, y1, x2, y2 = map(int, box)  # Get bounding box coordinates
                cv2.rectangle(frame, (x1, y1), (x2, y2), [0, 255, 0], 2)

                # Draw text with background showing object class and confidence
                draw_text_with_background(frame,
                                          f"{coco[int(cls)].capitalize()}, conf:{(conf)*100:0.2f}%",
                                          (x1, y1 - 10),
                                          cv2.FONT_HERSHEY_COMPLEX,
                                          0.6,
                                          (255, 255, 255),  # White text
                                          (0, 0, 0),  # Black background
                                          (0, 0, 255))  # Red border

                # Check if red light is on and if object is inside the ROI
                if is_region_light(frame, RedLight):
                    print("Violation detected.")
                    if cv2.pointPolygonTest(ROI, (x1, y1), False) >= 0 or cv2.pointPolygonTest(ROI, (x2, y2), False) >= 0:
                        # Add the violation text at the top-left corner
                        cv2.putText(frame,
                                    "The car violated the traffic signal!",  # Text message
                                    (50, 50),  # Position
                                    cv2.FONT_HERSHEY_SIMPLEX,  # Font
                                    1.0,  # Font scale
                                    (0, 0, 255),  # Red color
                                    2)  # Thickness

                        draw_text_with_background(frame,
                                                  f"The {coco[int(cls)].capitalize()} violated the traffic signal.",
                                                  (10, 30),
                                                  cv2.FONT_HERSHEY_COMPLEX,
                                                  0.6,
                                                  (255, 255, 255),  # White text
                                                  (0, 0, 0),  # Black background
                                                  (0, 0, 255))  # Red border

                        # Highlight the violation by redrawing the ROI and the object's bounding box in red
                        cv2.polylines(frame, [ROI], True, [0, 0, 255], 2)  # Red ROI for violation
                        cv2.rectangle(frame, (x1, y1), (x2, y2), [0, 0, 255], 2)  # Red bounding box for violation
    # Display the frame
    # cv2_imshow(frame)
    video_writer.write(frame)

    if cv2.waitKey(1) == 27:  # Press 'Esc' to quit
        break

# Release the video capture object and close windows
cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 1020.5ms
Speed: 14.9ms preprocess, 1020.5ms inference, 16.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 879.0ms
Speed: 4.1ms preprocess, 879.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 865.2ms
Speed: 3.1ms preprocess, 865.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 866.0ms
Speed: 3.1ms preprocess, 866.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 868.6ms
Speed: 3.1ms preprocess, 868.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 845.2ms
Speed: 3.1ms preprocess, 845.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 849.1ms
Speed: 3.0ms preprocess, 849.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 853.5ms
Speed: 3.0ms p

In [28]:
# Run validation on your test dataset
metrics = model.val(data='coco128.yaml')

# Print evaluation metrics
print(metrics)


Ultralytics YOLOv8.2.87 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8m summary (fused): 218 layers, 25,886,080 parameters, 0 gradients, 78.9 GFLOPs

Dataset 'coco128.yaml' images not found ⚠️, missing path '/content/datasets/coco128/images/train2017'


100%|██████████| 6.66M/6.66M [00:00<00:00, 77.5MB/s]
Unzipping /content/datasets/coco128.zip to /content/datasets/coco128...: 100%|██████████| 263/263 [00:00<00:00, 1889.03file/s]

Dataset download success ✅ (0.7s), saved to /content/datasets



100%|██████████| 755k/755k [00:00<00:00, 15.3MB/s]
val: Scanning /content/datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 1712.27it/s]

val: New cache created: /content/datasets/coco128/labels/train2017.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [03:47<00:00, 28.42s/it]


                   all        128        929      0.712       0.73      0.784      0.613
                person         61        254       0.83       0.74      0.856      0.654
               bicycle          3          6          1      0.625      0.789      0.579
                   car         12         46      0.757      0.339      0.552      0.306
            motorcycle          4          5      0.761          1      0.962      0.805
              airplane          5          6      0.826          1      0.995      0.928
                   bus          5          7      0.753      0.714      0.837      0.719
                 train          3          3      0.816          1      0.995      0.995
                 truck          5         12      0.743      0.485      0.588      0.337
                  boat          2          6          1      0.655      0.737      0.565
         traffic light          4         14      0.803      0.292      0.455       0.25
             stop sig

## Save and Submit Your Work

In [15]:
model.save('yolov8_model.h5')